In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
from tqdm import tqdm
from google.colab import drive
import numpy as np
import pandas as pd 
from pandas import DataFrame
import spacy
import operator

In [0]:
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Read the file that contain Title token and Description token
df_train_X = pd.read_csv("gdrive/My Drive/5500/job-salary-prediction/Train_X_modified.csv")

In [0]:
df_train_X

,Unnamed: 0,Title,Token Title,FullDescription,Token Description,LocationNormalized,ContractType,ContractTime,Company,Category
0,0,Engineering Systems Analyst,"['engineering', 'systems', 'analyst']",Engineering Systems Analyst Dorking Surrey Sal...,"['engineering', 'systems', 'analyst', 'dorking...",Dorking,NaN,permanent,Gregory Martin International,Engineering Jobs
1,1,Stress Engineer Glasgow,"['stress', 'engineer', 'glasgow']",Stress Engineer Glasgow Salary **** to **** We...,"['stress', 'engineer', 'glasgow', 'salary', 'c...",Glasgow,NaN,permanent,Gregory Martin International,Engineering Jobs
2,2,Modelling and simulation analyst,"['modelling', 'simulation', 'analyst']",Mathematical Modeller / Simulation Analyst / O...,"['mathematical', 'modeller', 'simulation', 'an...",Hampshire,NaN,permanent,Gregory Martin International,Engineering Jobs
3,3,Engineering Systems Analyst / Mathematical Mod...,"['engineering', 'systems', 'analyst', 'mathema...",Engineering Systems Analyst / Mathematical Mod...,"['engineering', 'systems', 'analyst', 'mathema...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs
4,4,"Pioneer, Miser Engineering Systems Analyst","['pioneer', 'miser', 'engineering', 'systems',...","Pioneer, Miser Engineering Systems Analyst Do...","['pioneer', 'miser', ' ', 'engineering', 'syst...",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs
...,...,...,...,...,...,...,...,...,...,...
244763,244763,TEACHER OF SCIENCE,"['teacher', 'science']",Position: Qualified Teacher Subject/Specialism...,"['position', 'qualified', 'teacher', 'subject'...",Swindon,NaN,contract,NaN,Teaching Jobs
244764,244764,TEACHER OF BUSINESS STUDIES AND ICT,"['teacher', 'business', 'studies', 'ict']",Position: Qualified Teacher or NQT Subject/Spe...,"['position', 'qualified', 'teacher', 'nqt', 's...",Swindon,NaN,contract,NaN,Teaching Jobs
244765,244765,ENGLISH TEACHER,"['english', 'teacher']",Position: Qualified Teacher Subject/Specialism...,"['position', 'qualified', 'teacher', 'subject'...",Swindon,NaN,contract,NaN,Teaching Jobs
244766,244766,SUPPLY TEACHERS,"['supply', 'teacher']",Position: Qualified Teacher Subject/Specialism...,"['position', 'qualified', 'teacher', 'subject'...",Wiltshire,NaN,contract,NaN,Teaching Jobs


In [0]:
# Make a dictionary, contain the frequency of each words
def list_to_count_dic(Text_List):
  vol_dic = {}
  for i in tqdm(Text_List):
    spited_i=i[1:-1].split(', ')
    for j in spited_i:
      if j[1:-1] != ' ':
        if j[1:-1] not in vol_dic:
          vol_dic[j[1:-1]] = 1
        else:
          vol_dic[j[1:-1]] += 1
  
  return vol_dic

# Reorder the dictionary above from most frequent word to least frequent word, and then take a "vol_size" number of most frequent words as the vocabulary
def build_vocabulary(dic, vol_size):
  sorted_dic = dict(sorted(dic.items(), key=operator.itemgetter(1),reverse=True))
  vol = list(sorted_dic)[:vol_size]
  return vol

In [0]:
# Build volcabulary for description
description_dic = list_to_count_dic(df_train_X["Token Description"])
train_description_vol = build_vocabulary(description_dic, 2000)

100%|██████████| 244768/244768 [00:18<00:00, 13029.93it/s]


In [0]:
# Build volcabulary for title
title_dic = list_to_count_dic(df_train_X["Token Title"])
train_title_vol = build_vocabulary(title_dic, 200)

Bag of Words: representation of text that describes the occurrence of words within a document.

Example:

Vocabulary: ["today", "sunny", "rainy"]

text: Today is a sunny day. -> vector: [1, 1, 0]

In [0]:
# Transform each text token to a bag-of-words vector

def token_to_bag_of_words_vector(text_token, vol):
  vec = []
  spited_token=text_token[1:-1].split(', ')
  new_token = []
  for j in spited_token:
    if j[1:-1] != ' ':
      new_token.append(j[1:-1])
  #print(new_token)
  for i in vol:
    if i in new_token:
      #print("yes")
      vec.append(1)
    else:
      vec.append(0)
  return vec

In [0]:
# Convert the array of title token to Bag of Words vector
train_title_BoG = []
for i in tqdm(df_train_X["Token Title"]):
  token_vec = token_to_bag_of_words_vector(i,train_title_vol)
  train_title_BoG.append(token_vec)

In [0]:
# Convert the array of description token to Bag of Words vector
train_description_BoG = []
for i in tqdm(df_train_X["Token Description"]):
  token_vec = token_to_bag_of_words_vector(i,train_description_vol)
  train_description_BoG.append(token_vec)